In [16]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

import huggingface_hub

import yaml

In [18]:
mlp_config_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "torch_mlp_config.yaml")

mlp_weights_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "mlp_weights.pt")

wavenet_config_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "wavenet_config.yaml")

wavenet_weights_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "wavenet_weights.pt")

gpt_nano_config_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "gpt_config.yaml")

gpt_nano_weights_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "gpt_weights.pt")

In [19]:
with open(mlp_config_path, 'r') as file:
    mlp_config = yaml.safe_load(file)

with open(wavenet_config_path, 'r') as file:
    wavenet_config = yaml.safe_load(file)
    
with open(gpt_nano_config_path, 'r') as file:
    gpt_nano_config = yaml.safe_load(file)

In [20]:
class MLP(nn.Module):
    def __init__(self, num_char, hidden_nodes, embeddings, window, num_layers):   
        super(MLP, self).__init__()
        
        self.window = window
        self.hidden_nodes = hidden_nodes
        self.embeddings = embeddings
        
        self.C = nn.Parameter(torch.randn((num_char, embeddings)) * 0.1, requires_grad=True)
        
        self.first = nn.Linear(embeddings*window, hidden_nodes)

        self.layers = nn.Sequential()
        for i in range(num_layers):
            self.layers = self.layers.extend(nn.Sequential(
                nn.Linear(hidden_nodes, hidden_nodes, bias=False),
                nn.BatchNorm1d(hidden_nodes),
                nn.Tanh()))

        self.final = nn.Linear(hidden_nodes, num_char)
        
    def forward(self, x):
        x = self.C[x]
        x = self.first(x.view(-1, self.window*self.embeddings))
        
        x = self.layers(x)

        x = self.final(x)
        return x
    
    def sample_char(self, x):
        logits = self(x)
        probs = F.softmax(logits, dim=1)
        return torch.multinomial(probs, num_samples=1).item()
     
mlp = MLP(mlp_config['num_char'], 
          mlp_config['hidden_nodes'], 
          mlp_config['embeddings'], 
          mlp_config['window'], 
          mlp_config['num_layers'])

mlp.load_state_dict(torch.load(mlp_weights_path))
mlp.eval()


MLP(
  (first): Linear(in_features=30, out_features=200, bias=True)
  (layers): Sequential(
    (0): Linear(in_features=200, out_features=200, bias=False)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Linear(in_features=200, out_features=200, bias=False)
    (4): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Tanh()
    (6): Linear(in_features=200, out_features=200, bias=False)
    (7): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Tanh()
    (9): Linear(in_features=200, out_features=200, bias=False)
    (10): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Tanh()
    (12): Linear(in_features=200, out_features=200, bias=False)
    (13): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Tanh()
    (15): Linear(in_features=200, out_features=200, bias=F

In [21]:
class WaveNet(nn.Module):
    def __init__(self, num_char, hidden_nodes, embeddings, window, num_layers):   
        super(WaveNet, self).__init__()
        
        self.window = window
        self.hidden_nodes = hidden_nodes
        self.embeddings = embeddings        
        
        self.layers = nn.Sequential(
            nn.Embedding(num_char, embeddings)
        )
        
        for i in range(num_layers):
            if i == 0:
                nodes = window
            else:
                nodes = hidden_nodes
                
            self.layers = self.layers.extend(nn.Sequential(
                nn.Conv1d(nodes, hidden_nodes, kernel_size=2, stride=1, bias=False),
                nn.BatchNorm1d(hidden_nodes),
                nn.Tanh()))
            
        self.layers = self.layers.extend(nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_nodes*(embeddings-num_layers), num_char)
        ))
        
    def forward(self, x):
        return self.layers(x)
    
    def sample_char(self, x):
        logits = self(x)
        probs = F.softmax(logits, dim=1)
        return torch.multinomial(probs, num_samples=1).item()
    
wavenet = WaveNet(wavenet_config['num_char'], 
                  wavenet_config['hidden_nodes'], 
                  wavenet_config['embeddings'], 
                  wavenet_config['window'], 
                  wavenet_config['num_layers'])
wavenet.load_state_dict(torch.load(wavenet_weights_path))
wavenet.eval()

WaveNet(
  (layers): Sequential(
    (0): Embedding(30, 10)
    (1): Conv1d(8, 100, kernel_size=(2,), stride=(1,), bias=False)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Tanh()
    (4): Conv1d(100, 100, kernel_size=(2,), stride=(1,), bias=False)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
    (7): Conv1d(100, 100, kernel_size=(2,), stride=(1,), bias=False)
    (8): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Tanh()
    (10): Conv1d(100, 100, kernel_size=(2,), stride=(1,), bias=False)
    (11): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Tanh()
    (13): Conv1d(100, 100, kernel_size=(2,), stride=(1,), bias=False)
    (14): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Tanh()
    (16): Flatten(start_dim=1, end_dim=-1)
    (17): Linear(i

In [22]:
class NewGELU(nn.Module):
    """
    Gaussian Error Linear Units (GELU) paper: https://arxiv.org/abs/1606.08415
    """
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

class GptAttention(nn.Module):
    """
    For this attention module k = v = q are all the same.
    It's for encoder/decoder only transfomers.
    """
    def __init__(self, config):
        super(GptAttention, self).__init__()
        self.config = config

        assert self.config["d_model"] % self.config["heads"] == 0
        self.heads = self.config["heads"]

        self.w_attn = nn.Linear(self.config["d_model"], 3*self.config["d_model"])
        self.head = nn.Linear(self.config["d_model"], self.config["d_model"])

        self.attn_dropout = nn.Dropout(config["attn_pdrop"])
        self.resid_dropout = nn.Dropout(config["resid_pdrop"])

        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer(
            "bias", 
            torch.tril(
                torch.ones(
                    self.config["window"], 
                    self.config["window"])
                ).view(1, 1, self.config["window"], self.config["window"])
        )
    
    def forward(self, x):
        B, window, embs = x.shape

        q, v, k = self.w_attn(x).split(self.config["d_model"], dim=2)

        # (B, heads, window, embs)
        q = q.view(
            B, 
            window, 
            self.config["heads"], 
            embs // self.config["heads"]
        ).transpose(1, 2)
        k = k.view(
            B, 
            window, 
            self.config["heads"], 
            embs // self.config["heads"]
        ).transpose(1, 2)
        v = v.view(
            B, 
            window, 
            self.config["heads"], 
            embs // self.config["heads"]
        ).transpose(1, 2)
        
        # Self-attend: (B, heads, window, embs) x (B, heads, embs, window) -> (B, heads, window, window)
        scores = q @ k.transpose(-2, -1) / math.sqrt(k.size(-1))
        mask = scores.masked_fill(self.bias[:,:,:window,:window] == 0, float('-inf'))
        probs = F.softmax(mask, dim=-1)
        attn = self.attn_dropout(probs)
        attn = probs @ v
        attn = attn.transpose(1, 2).contiguous().view(B, window, embs)

        return self.resid_dropout(self.head(attn))

class FeedForward(nn.Module):
    def __init__(self, config):
        super(FeedForward, self).__init__()
        self.l1 = nn.Linear(config["d_model"], 4*config["d_model"])
        self.l2 = nn.Linear(4*config["d_model"], config["d_model"])
        self.dropout = nn.Dropout(config["resid_pdrop"])

    def forward(self, x):
        x = NewGELU()(self.l1(x))
        return self.dropout(self.l2(x))

class Block(nn.Module):
    def __init__(self, config):
        super(Block, self).__init__()
        self.attn = GptAttention(config)
        self.norm1 = nn.LayerNorm(config["d_model"])
        self.ff = FeedForward(config)
        self.norm2 = nn.LayerNorm(config["d_model"])

    def forward(self, x):
        x = self.norm1(x + self.attn(x))
        x = self.norm2(x + self.ff(x))
        return x

class GPT(nn.Module):
    def __init__(self, config):
        super(GPT, self).__init__()
        self.config = config

        self.vocab_emb = nn.Embedding(self.config["vocab"], self.config["d_model"])
        self.pos_emb = nn.Embedding(self.config["window"], self.config["d_model"])
        self.emb_dropout = nn.Dropout(config["embd_pdrop"])

        self.blocks = nn.ModuleList([Block(self.config) for _ in range(self.config["blocks"])])
        self.head_layer_norm = nn.LayerNorm(config["d_model"])
        self.head = nn.Linear(self.config["d_model"], self.config["vocab"])

    def forward(self, x):
        vocab_emb = self.vocab_emb(x)
        pos_emb = self.pos_emb(torch.arange(0, x.shape[1], dtype=torch.long, device=x.device))

        x = self.emb_dropout(vocab_emb + pos_emb)

        for b in self.blocks:
            x = b(x)

        x = self.head_layer_norm(x)
        x = self.head(x)

        return x

    def configure_opt(self):
        p_decay = set()
        p_no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name
                # random note: because named_modules and named_parameters are recursive
                # we will see the same tensors p many many times. but doing it this way
                # allows us to know which parent module any tensor p belongs to...
                if pn.endswith('bias'):
                    # all biases will not be decayed
                    p_no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    p_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    p_no_decay.add(fpn)

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = p_decay & p_no_decay
        union_params = p_decay | p_no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )

        # create the pytorch optimizer object
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(p_decay))], "weight_decay": self.config["weight_decay"]},
            {"params": [param_dict[pn] for pn in sorted(list(p_no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(
            optim_groups, 
            lr=self.config["lr"], 
            betas=(self.config["b1"], self.config["b2"])
        )
        return optimizer

    def sample_char(self, x):
        logits = self(x)
        probs = F.softmax(logits[:,-1,:], dim=1)
        return torch.multinomial(probs, num_samples=1).item()

gpt_nano = GPT(gpt_nano_config)
gpt_nano.load_state_dict(torch.load(gpt_nano_weights_path))
gpt_nano.eval()

GPT(
  (vocab_emb): Embedding(30, 48)
  (pos_emb): Embedding(32, 48)
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (attn): GptAttention(
        (w_attn): Linear(in_features=48, out_features=144, bias=True)
        (head): Linear(in_features=48, out_features=48, bias=True)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
      (ff): FeedForward(
        (l1): Linear(in_features=48, out_features=192, bias=True)
        (l2): Linear(in_features=192, out_features=48, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
    )
    (1): Block(
      (attn): GptAttention(
        (w_attn): Linear(in_features=48, out_features=144, bias=True)
        (head): Linear(in_features=48, out_features=48, bias=True)
        (attn_dropout): D

In [23]:
next(mlp.parameters()).device, next(wavenet.parameters()).device, next(gpt.parameters())

(device(type='cpu'),
 device(type='cpu'),
 Parameter containing:
 tensor([[-0.5183,  0.0326,  1.4677,  ...,  1.1205,  0.8975,  1.3533],
         [ 0.8270, -1.1276,  1.2416,  ...,  2.4547,  1.7785,  2.9467],
         [ 0.0877,  0.1408,  0.9961,  ...,  0.0688,  0.2418,  0.8137],
         ...,
         [ 0.3698, -0.7645,  0.8274,  ...,  0.3665, -0.1461,  0.3645],
         [ 1.1430, -1.0511,  1.2755,  ...,  0.3180, -0.1909, -0.3718],
         [-0.9094, -1.5168,  1.5178,  ...,  0.5179,  0.5387,  1.8614]],
        requires_grad=True))

In [24]:
import gradio as gr

def generate_names(name_start, number_of_names, model):
    if model == "MLP":
        stoi = mlp_config['stoi']
        window = mlp_config['window']
    elif model == "WaveNet":
        stoi = wavenet_config['stoi']
        window = wavenet_config['window']
    elif model == "GPT Nano":
        stoi = gpt_nano_config['stoi']
        window = gpt_nano_config['window']
    itos = {s:i for i,s in stoi.items()}

    names = ""
    for _ in range((int)(number_of_names)):
    
        # Initialize name with user input
        name = ""
        context = [0] * window
        for c in name_start.lower():
            name += c
            context = context[1:] + [stoi[c]]

        # Run inference to finish off the name
        while True:
            x = torch.tensor(context).view(1, -1)
            if model == "MLP":
                ix = mlp.sample_char(x)
            elif model == "WaveNet":
                ix = wavenet.sample_char(x)
            elif model == "GPT Nano":
                ix = gpt_nano.sample_char(x)
                
            context = context[1:] + [ix]
            name += itos[ix]
                
            if ix == 0:
                break
            
        names += name + "\n"
        
    return names

demo = gr.Interface(
    fn=generate_names,
    inputs=[
        gr.Textbox(placeholder="Start name with..."),
        gr.Number(value=5),
        gr.Dropdown(["MLP", "WaveNet", "GPT Nano"], value="GPT Nano"),
    ],
    outputs="text",
)
demo.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
